<a href="https://colab.research.google.com/github/debemdeboas/pucrs-aprendizado-de-maquina-t2/blob/master/PUCRS_Aprendizado_de_M%C3%A1quina_T2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Download and extract our dataset.

This will download a tarred file and extract it into `dist`. Then, we're renaming it to `animes/`.
This directory contains the following files:
- `animes.csv`, the CSV containing anime IDs, URLs, titles, genres, and poster path
- `animes.pkl`, serialized (pickled) list of `Anime` instances. This isn't used by this notebook
- `images/`, a directory that contains all of our anime posters as `images/<mal_id>.jpg` files


In [1]:
import requests

ds = requests.get("https://public-s3.debem.dev/anime_dataset.tar.xz", allow_redirects=True)

with open("anime_dataset.tar.xz", "wb") as f:
    f.write(ds.content)

!tar xf anime_dataset.tar.xz

mv: cannot stat 'dist': No such file or directory


Load the dataset into memory

In [37]:
import pandas as pd

df = pd.read_csv("animes.csv")

In [38]:
df = df.dropna()
df["genres"] = df["genres"].str.split('|')

In [54]:
all_genres_to_idx = dict()
all_genres = list()
for gl in df.genres:
    for g in gl:
        if g not in all_genres_to_idx:
            all_genres.append(g)
            all_genres_to_idx[g] = len(all_genres) - 1

In [62]:
from torch.utils.data.dataset import Dataset
from PIL import Image

class PosterMultiLabelDataset(Dataset):
    def __init__(self, df: pd.DataFrame, *args, **kwargs):
        self.df = df
        super().__init__(*args, **kwargs)

    def __len__(self) -> int:
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.loc[idx]
        img = Image.open(row["img_path"])

        return {
            "image": img,
            # "genres": {g: g in row.genres for g in all_genres}
            "genres": torch.Tensor([1 if g in df.loc[0].genres else 0 for g in all_genres])
        }

Now let's get some transfer learning done.

We'll use a pre-trained convolutional network to analyze the posters to define which genres a given anime belongs to.
Each anime can belong to any number of genres.


In [64]:
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
import torch.nn as nn
import seaborn as sns
import numpy as np
import torchvision
import random
import torch

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

def validation(model, loader, criterion):
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for data in loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs,labels)
            val_loss +=loss
    return val_loss/len(loader)

def train(model, trainloader, testloader, optimizer, criterion, epochs):
    for epoch in range(epochs):
        model.train()
        running_loss = 0
        for data in tqdm(trainloader):
            images, labels = data
            model.zero_grad()
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        val_loss = validation(model, testloader, criterion)
        print(f'Epoch: {epoch+1} | Loss: {running_loss/len(trainloader)} | Val Loss: {val_loss}')

def accuracy(model, loader):
    model.eval()
    corrected = 0
    total = 0
    with torch.no_grad():
        for data in loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _,predicted = torch.max(outputs, 1)
            total += labels.size(0)
            corrected += (predicted == labels).sum().item()
    return corrected * 100 // total

def confusion_matrix(model, loader):
    model.eval()
    confusion_matrix = np.zeros((2,2))
    with torch.no_grad():
        for data in loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _,predicted = torch.max(outputs, 1)
            for i in range(labels.size(0)):
                confusion_matrix[labels[i].item()][predicted[i].item()] += 1
    ax = sns.heatmap(confusion_matrix, annot=True, cmap='Blues', fmt='g')
    ax.set_xlabel('Predicted')
    ax.set_ylabel('Label')
    return ax

In [65]:
def imshow(img):
    plt.figure(figsize=(20,8))
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

img_size = (256,256)
transformations = transforms.Compose([transforms.Resize(img_size)])
